## Conv2D: Исследование аргументов (с интерактивным Playground)
**🇷🇺 (RU)**: В этом ноутбуке мы исследуем, как различные аргументы слоя `Conv2D` в TensorFlow/Keras влияют на результат свертки. Сначала мы рассмотрим каждый важный параметр шаг за шагом, а в конце вы найдете интерактивную песочницу, где сможете поэкспериментировать самостоятельно.

---

**🇬🇧 (EN)**: In this notebook, we'll explore how different arguments of the `Conv2D` layer in TensorFlow/Keras affect the convolution output. First, we will look at each important parameter step-by-step, and at the end, you will find an interactive sandbox where you can experiment on your own.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from typing import Tuple, Optional
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
input_image = np.arange(1, 26).reshape((5, 5))
print("Наша 'картинка' (входные данные):\n")
print(input_image)

#### Вспомогательная функция для визуализации

**🇷🇺 (RU)**: Чтобы не повторять код, мы создадим одну функцию, которая будет проводить эксперимент: создавать модель, применять свертку и рисовать три картинки: **входное изображение**, **ядро свертки** и **результат (карту признаков)**.

---

**🇬🇧 (EN)**: To avoid repeating code, we will create a single function to run the experiment: it will create a model, apply the convolution, and draw three images: the **input image**, the **convolution kernel**, and the **result (feature map)**.

In [ ]:
def run_convolution_experiment(
    input_data: np.ndarray,
    kernel_size: Tuple[int, int] = (3, 3),
    strides: Tuple[int, int] = (1, 1),
    padding: str = "valid",
    dilation_rate: Tuple[int, int] = (1, 1),
    custom_kernel: Optional[np.ndarray] = None,
):
    """Проводит один эксперимент со сверткой и визуализирует результат."""
    tf.keras.backend.clear_session()
    input_shape = input_data.shape

    if len(input_shape) == 2:
        input_data_ch = np.expand_dims(input_data, axis=-1)
    else:
        input_data_ch = input_data
    
    input_data_batch = np.expand_dims(input_data_ch, axis=0)

    model = Sequential()
    conv_layer = Conv2D(
        filters=1,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        dilation_rate=dilation_rate,
        use_bias=False,
        input_shape=input_data_ch.shape,
    )
    model.add(conv_layer)

    if custom_kernel is not None:
        keras_kernel = np.reshape(custom_kernel, (*kernel_size, 1, 1))
        model.set_weights([keras_kernel])

    output_data = model.predict(input_data_batch, verbose=0)

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    fig.suptitle(f'Kernel: {kernel_size}, Strides: {strides}, Padding: "{padding}", Dilation: {dilation_rate}', fontsize=14)

    ax1 = axs[0]
    im1 = ax1.imshow(input_data, cmap="viridis", interpolation="nearest")
    ax1.set_title(f"Input\nShape: {input_data.shape}")
    fig.colorbar(im1, ax=ax1)

    kernel_weights = model.get_weights()[0]
    if len(kernel_weights.shape) == 4 and kernel_weights.shape[2] == 1 and kernel_weights.shape[3] == 1:
        kernel_to_show = kernel_weights[:, :, 0, 0]
    else:
        kernel_to_show = kernel_weights.squeeze()

    ax2 = axs[1]
    im2 = ax2.imshow(kernel_to_show, cmap="inferno", interpolation="nearest")
    ax2.set_title(f"Kernel\nShape: {kernel_to_show.shape}")
    fig.colorbar(im2, ax=ax2)

    output_squeezed = output_data.squeeze()
    ax3 = axs[2]
    if output_squeezed.shape == ():
        output_to_show = np.array([[output_squeezed]])
    else:
        output_to_show = output_squeezed
    im3 = ax3.imshow(output_to_show, cmap="viridis", interpolation="nearest")
    ax3.set_title(f"Output\nShape: {output_squeezed.shape}")
    fig.colorbar(im3, ax=ax3)

    for ax in axs:
        ax.grid(which='both', color='white', linestyle='-', linewidth=0.5)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

### Часть 1: Пошаговое исследование
---

#### `kernel_size`
**🇷🇺 (RU)**: `kernel_size` — это размер «окна», которое скользит по изображению. Давайте используем ядро 3x3, заполненное единицами. Каждое значение в итоговой карте будет суммой всех значений в окне 3x3.

**🇬🇧 (EN)**: `kernel_size` is the size of the "window" that slides over the image. Let's use a 3x3 kernel filled with ones. Each value in the resulting feature map will be the sum of all values in the 3x3 window.

In [ ]:
kernel_3x3 = np.ones((3, 3))
run_convolution_experiment(input_image, kernel_size=(3, 3), custom_kernel=kernel_3x3)

#### `strides`
**🇷🇺 (RU)**: `strides` — это шаг, с которым ядро двигается по изображению. Шаг (2, 2) означает, что ядро будет перепрыгивать через один пиксель. Посмотрите, как это уменьшает размер итоговой карты признаков.

**🇬🇧 (EN)**: `strides` is the step size with which the kernel moves across the image. A stride of (2, 2) means the kernel will skip over one pixel at a time. Notice how this reduces the size of the final feature map.

In [ ]:
run_convolution_experiment(input_image, kernel_size=(3, 3), strides=(2, 2), custom_kernel=kernel_3x3)

#### `padding`
**🇷🇺 (RU)**: `padding` добавляет «рамку» из нулей вокруг изображения. Если `padding='same'`, рамка будет такого размера, чтобы итоговая карта признаков была того же размера, что и входная (при шаге 1). Это полезно, чтобы не терять информацию по краям.

**🇬🇧 (EN)**: `padding` adds a "border" of zeros around the image. If `padding='same'`, the border will be sized so that the output feature map is the same size as the input (with a stride of 1). This is useful for not losing information at the edges.

In [ ]:
run_convolution_experiment(input_image, kernel_size=(3, 3), padding='same', custom_kernel=kernel_3x3)

#### `dilation_rate`
**🇷🇺 (RU)**: `dilation_rate` — это «разрежение» ядра. Значение (2, 2) означает, что между элементами ядра будет один пустой пиксель. Это позволяет ядру «видеть» большую область, не увеличивая количество параметров.

**🇬🇧 (EN)**: `dilation_rate` refers to the "sparseness" of the kernel. A value of (2, 2) means there will be one empty pixel between the elements of the kernel. This allows the kernel to "see" a larger area without increasing the number of parameters.

In [ ]:
run_convolution_experiment(input_image, kernel_size=(3, 3), dilation_rate=(2, 2), custom_kernel=kernel_3x3)

### Часть 2: Интерактивный Playground
---
**🇷🇺 (RU)**: Теперь ваша очередь! Двигайте ползунки и выбирайте значения из списков, чтобы увидеть, как меняется результат свертки.

**🇬🇧 (EN)**: Now it's your turn! Move the sliders and select values from the dropdowns to see how the convolution output changes.

In [ ]:
@interact(
    kernel_size_h=widgets.IntSlider(min=1, max=5, step=2, value=3, description='Kernel Height:'),
    kernel_size_w=widgets.IntSlider(min=1, max=5, step=2, value=3, description='Kernel Width:'),
    stride_h=widgets.IntSlider(min=1, max=5, step=1, value=1, description='Stride Height:'),
    stride_w=widgets.IntSlider(min=1, max=5, step=1, value=1, description='Stride Width:'),
    padding=widgets.Dropdown(options=['valid', 'same'], value='valid', description='Padding:'),
    dilation_h=widgets.IntSlider(min=1, max=3, step=1, value=1, description='Dilation Height:'),
    dilation_w=widgets.IntSlider(min=1, max=3, step=1, value=1, description='Dilation Width:')
)
def interactive_convolution_visualizer(kernel_size_h, kernel_size_w, stride_h, stride_w, padding, dilation_h, dilation_w):
    """A wrapper function to connect widgets to our experiment function."""
    if padding == 'valid':
        if kernel_size_h > input_image.shape[0] or kernel_size_w > input_image.shape[1]:
            print("Error: Kernel size cannot be larger than the input image with 'valid' padding.")
            return
            
    custom_kernel = np.ones((kernel_size_h, kernel_size_w))
    
    run_convolution_experiment(
        input_image,
        kernel_size=(kernel_size_h, kernel_size_w),
        strides=(stride_h, stride_w),
        padding=padding,
        dilation_rate=(dilation_h, dilation_w),
        custom_kernel=custom_kernel
    )